In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
import numpy as np
from tqdm import tqdm

# Prepare product images for data generation

In [ ]:
def resize(image):
    width, height = image.size
    
    background = Image.new('RGBA', (int(width+500), int(height+500)), (255, 255, 255, 255))
    offset = (250, 250)
    background.paste(image, offset)
    return background.convert('RGB')

def no_mask_pipeline(directory, directory_out, prints=False):
    for filename in os.listdir(directory):
        image_ori = Image.open(os.path.join(directory, filename), 'r')
        img = resize(image_ori)
        filename = filename.split('.')[0] + '.png'
        img.save(os.path.join(directory_out, filename))
        if prints:
            fig = plt.imshow(img)
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False)
            plt.show()
        

#directory = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/objects/fridges/'
#directory_out = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/objects/out/'
#no_mask_pipeline(directory, directory_out)

In [ ]:
def resize_with_alpha(image):
    red, green, blue, alpha = image.split()
    contours, _ = cv2.findContours(cv2.dilate(np.array(alpha), np.ones((10, 10), np.uint8), iterations=1), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);
    width, height = image.size
    if cv2.contourArea(contours[0])/(width*height) > 0.8:
        print('Extra border generated')
        background = Image.new('RGBA', (width*2, height*2), (255, 255, 255, 0))
        offset = (round(width/2), round(height/2))
    else:
        background = Image.new('RGBA', (int(width*1.5), int(height*1.5)), (255, 255, 255, 0))
        offset = (round(width/4), round(height/4))
    background.paste(image, offset)
    return background.convert('RGBA')

def reduce_size(img):
    red, green, blue, alpha = img.split()
    
    kernel = np.ones((30, 30), np.uint8)
    alpha_dilated = cv2.dilate(np.array(alpha), kernel, iterations=2)
    contours, _ = cv2.findContours(alpha_dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);
    if len(contours) > 1:
        print('Too many contours')
        return img
    else:
        x,y,w,h = cv2.boundingRect(contours[0])
        xmin = x
        xmax = x+w
        ymin = y
        ymax = y+h
        xmin -= int(0.15 * (xmax - xmin))
        xmax += int(0.15 * (xmax - xmin))
        ymin -= int(0.15 * (ymax - ymin))
        ymax += int(0.15 * (ymax - ymin))
        croped = np.array(img)[ymin:ymax, xmin:xmax]
        return Image.fromarray(croped)


def with_mask_pipeline(directory, directory_out):
    for filename in os.listdir(directory):
        f, axarr = plt.subplots(1,7)
        image_ori = Image.open(os.path.join(directory, filename), 'r')
        print(filename)
        axarr[0].imshow(image_ori)
        axarr[0].get_xaxis().set_visible(False)
        axarr[0].get_yaxis().set_visible(False)
        axarr[1].imshow(Image.merge('RGB', image_ori.split()[:-1]))
        axarr[1].get_xaxis().set_visible(False)
        axarr[1].get_yaxis().set_visible(False)
        axarr[2].imshow(image_ori.split()[-1])
        axarr[2].get_xaxis().set_visible(False)
        axarr[2].get_yaxis().set_visible(False)
        #red, green, blue, alpha = image_ori.split()
        img = resize_with_alpha(image_ori)
        #img = fix_alpha_layer(img)

        axarr[3].imshow(img)
        axarr[3].get_xaxis().set_visible(False)
        axarr[3].get_yaxis().set_visible(False)
        img = reduce_size(img)
        img.save(os.path.join(directory_out, filename))
        axarr[4].imshow(img)
        axarr[4].get_xaxis().set_visible(False)
        axarr[4].get_yaxis().set_visible(False)
        axarr[5].imshow(Image.merge('RGB', img.split()[:-1]))
        axarr[5].get_xaxis().set_visible(False)
        axarr[5].get_yaxis().set_visible(False)
        axarr[6].imshow(img.split()[-1])
        axarr[6].get_xaxis().set_visible(False)
        axarr[6].get_yaxis().set_visible(False)
        plt.show()
        
        mask = np.array(img)[:, :, 3]

        f, axarr = plt.subplots(1, 1, figsize=(15, 15))
        axarr.imshow(mask)
        axarr.get_xaxis().set_visible(False)
        axarr.get_yaxis().set_visible(False)
        plt.show()

        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #if len(contours) != 1:
            #print('More then 1 contour')
        print(len(contours))
            #raise Exception('!!')
        #else:
        img = np.array(img)
        #img[:, :, 3] = mask
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGRA)
        cv2.imwrite(os.path.join(directory_out, filename), img)
        
directory = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/objects/volleyball/'
directory_out = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/objects/out/'
with_mask_pipeline(directory, directory_out)

###  Background

In [ ]:
def resize(image):
    width, height = image.size
    
    background = Image.new('RGBA', (int(width+200), int(height+200)), (180, 180, 180, 255))
    offset = (100, 100)
    background.paste(image, offset)
    return background.convert('RGB')

directory = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/backgrounds/'
directory_out = '/home/eduard/PycharmProjects/synth2real/imagegeneration/samples/holiday/out/'

for filename in os.listdir(directory):
    if "front" not in filename and "area" not in filename:
        image_ori = Image.open(os.path.join(directory, filename), 'r')
        img = resize(image_ori)
        filename = filename.split('.')[0] + '.png'
        img.save(os.path.join(directory_out, filename))
        imgplot = plt.imshow(img)
        plt.show()

# Experiments:

### Generate regularisation Backgrounds

In [ ]:
directory = '/home/eduard/Desktop/data/implantation/'
directory_out = '/home/eduard/Desktop/data/implantation/backgrounds_out/'


for counter in range(0, 10):
    image_ori = Image.open(os.path.join(directory, 'regularisation_base.png'), 'r')
    image_ori_mask = Image.open(os.path.join(directory, 'regularisation_base_placement-area.png'), 'r')
    mask = np.array(image_ori_mask)[:-1]
    
    width, height = image_ori.size
    img_base = np.array(Image.new("RGB", (width, height), (100+counter*10, 100+counter*10, 100+counter*10)))
    
    width, height = image_ori_mask.size
    img_mask = np.array(Image.new("RGBA", (width, height), (100+counter*10, 100+counter*10, 100+counter*10, 255)))
    img_mask[:-1] = mask
    
    filename = 'regularisation'+str(counter)
    cv2.imwrite(os.path.join(directory_out, filename+'.png'), img_base)
    cv2.imwrite(os.path.join(directory_out, filename+'_placement-area'+ '.png'), img_mask)
    
    imgplot = plt.imshow(img_mask)
    plt.show()
    counter += 1

### Prepare Handextracted images

In [ ]:
def prepare_handextracted(image):
    #imgplot = plt.imshow(image)
    #plt.show()
    #imgplot = plt.imshow(image[:,:,-1])
    #plt.show()
    cropped = get_masked(img)
    #imgplot = plt.imshow(cropped)
    #plt.show()
    mask = cropped[:,:,-1]
    imgplot = plt.imshow(mask)
    plt.show()
    
    cropped[mask!=255]=(255, 255, 255, 0)
    imgplot = plt.imshow(cropped)
    plt.show()
    return cropped

def get_masked(img):
    h, w, _ = img.shape
    center = h // 2, w // 2
    contours, _ = cv2.findContours(img[:,:,3], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    if cv2.contourArea(cnt) < 100 or len(contours) > 1:
        print('++++error- to small or too many contourArea')
        imgplot = plt.imshow(img[:,:,3])
        plt.show()
    x,y,w,h = cv2.boundingRect(cnt)
    #print(x,y,w,h)
    x,y = max(x-int(w*0.3),0), max(y-int(h*0.3),0)
    w,h = int(w*1.5), int(h*1.5)
    #print(x,y,w,h)
    crop_img = img[y:y+h, x:x+w]
    print(crop_img.shape)
    return crop_img
    
directory = '/home/eduard/Desktop/export'
directory_out = '/home/eduard/Desktop/export/out/'

counter = 0
for subdir, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(subdir, file)
        if ".png" in file_path and '/out/' not in file_path:
            print(file_path)
            img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
            if img.shape[-1] == 4:
                img_new = prepare_handextracted(img)
                #print(img_new.shape, os.path.join(directory_out, file))
                cv2.imwrite(os.path.join(directory_out, str(counter)+file), img_new)
                counter = +1

### Prepare 360° Data


In [ ]:
directory = '/home/eduard/Downloads/Daten'


obj_list = set()
counter = 0
for subdir, dirs, files in os.walk(directory):
    splitted = subdir.split('/')
    if len(splitted) > 5:
        obj_list.add(splitted[5])

obj_images_dict = {}
for obj in obj_list:
    directory_obj = directory + '/' + obj+ '/2D/2D-Data/Single_Single/V30/'
    #print(directory_obj)
    images_list = []
    for subdir, dirs, files in os.walk(directory_obj):
        #print(subdir)
        for file in files:
            if ".jpg" in file :
                file_path = os.path.join(subdir, file)
                #print(file)
                images_list.append(file_path)
    
    obj_images_dict[obj] = images_list

print(obj_images_dict)

extra_flip = ['20008116','20347123','4056489025085','4056489035596','4056489038436','4056489108450','4056489126621','4056489126799','4056489262183','4056489452348','9006900218751']

directory_out = '/home/eduard/Downloads/ProductData/'

for k,v in tqdm(obj_images_dict.items()):

    #os.mkdir(directory_out+k)
    if k == '4337185486494':
        for i in v:
            img = cv2.imread(i, cv2.IMREAD_UNCHANGED)
            

            if img.shape[0] < img.shape[1]:
                img = np.rot90(img)
                if k in extra_flip:
                    img = np.rot90(img)
                    img = np.rot90(img)

            cv2.imwrite(directory_out+k+'/'+i.split('/')[-1], img)


In [ ]:

directory = '/home/eduard/Downloads/ProductDataSorted/'


for subdir, dirs, files in tqdm(os.walk(directory)):

    if len(subdir.split('/')) == 7:

        os.mkdir(subdir+'P')
        no_mask_pipeline(subdir, subdir+'P')


In [ ]:
directory = '/home/eduard/Downloads/ProductDataSorted/'

for subdir, dirs, files in os.walk(directory):
        for file in files:
            if ".jpg" in file :
                file_path = os.path.join(subdir, file)
                new_name = os.path.join(subdir, file.split('_')[2] + '.jpg')
                print(file_path, new_name)
                os.rename(file_path, new_name)